In [2]:
import IPython.display as ipd
import librosa.display
import numpy as np
import pandas as pd
from datetime import datetime
import tensorflow as tf

# Sources:
# https://www.youtube.com/watch?v=WJI-17MNpdE

In [3]:
start_time = datetime.now()

In [4]:
df = pd.read_csv('Dataset/participant_urbansound8k.csv')

In [5]:
labels = df.iloc[:7079, 3]
files = df.iloc[:, 0]
folder = df.iloc[:, 2]

In [6]:
tf.random.set_seed(310503)
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[13, 1000]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10000, activation="relu"))
model.add(tf.keras.layers.Dense(5000, activation="relu"))
model.add(tf.keras.layers.Dense(3512, activation="relu"))
model.add(tf.keras.layers.Dense(2312, activation="relu"))
model.add(tf.keras.layers.Dense(1512, activation="relu"))
model.add(tf.keras.layers.Dense(1055, activation="relu"))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 13000)             0         
                                                                 
 dense (Dense)               (None, 10000)             130010000 
                                                                 
 dense_1 (Dense)             (None, 5000)              50005000  
                                                                 
 dense_2 (Dense)             (None, 3512)              17563512  
                                                                 
 dense_3 (Dense)             (None, 2312)              8122056   
                                                                 
 dense_4 (Dense)             (None, 1512)              3497256   
                                                                 
 dense_5 (Dense)             (None, 1055)              1

In [8]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [13]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

data = []

for row in range(10000): #ICI QUON LOOP A TRAVERS LES 10 000 DONNÉES (REMPLACER 1 DANS RANGE QUAND ÇA MARCHE BIEN)
    try:
        audio_file = f"Dataset/fold{folder[row]}/{files[row]}"
        audio = ipd.Audio(audio_file)
        if librosa.get_duration(filename=audio_file) != 4.0:
            labels.pop(row)
            continue
        signal, sr = librosa.load(audio_file)
        mfccs = librosa.feature.mfcc(y=signal, n_mfcc=13, sr=sr)
        deltaMfccs = librosa.feature.delta(mfccs)
        delta2Mfccs = librosa.feature.delta(mfccs, order=2)
        compmfccs = np.concatenate(np.asarray((abs(np.concatenate(mfccs)),abs(np.concatenate(deltaMfccs)),abs(np.concatenate(delta2Mfccs)))))
        compmfccs = NormalizeData(compmfccs)
        data.append(compmfccs)
    except:
        print(row)
        print(f"Dataset/fold{folder[row]}/{files[row]}")

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

7180
Dataset/fold9/106955-6-0-0.wav
7181
Dataset/fold9/110389-3-0-0.wav
7186
Dataset/fold9/119420-3-0-0.wav
7187
Dataset/fold9/119420-3-1-0.wav
7216
Dataset/fold9/136399-6-0-0.wav
7217
Dataset/fold9/136399-6-1-0.wav
7218
Dataset/fold9/136399-6-2-0.wav
7219
Dataset/fold9/136399-6-3-0.wav
7220
Dataset/fold9/136399-6-4-0.wav
7221
Dataset/fold9/136399-6-5-0.wav
7222
Dataset/fold9/136399-6-6-0.wav
7223
Dataset/fold9/136399-6-7-0.wav
7224
Dataset/fold9/136399-6-8-0.wav
7225
Dataset/fold9/136399-6-9-0.wav
7240
Dataset/fold9/138468-1-0-0.wav
7269
Dataset/fold9/145683-6-0-0.wav
7270
Dataset/fold9/145683-6-1-0.wav
7271
Dataset/fold9/145683-6-2-0.wav
7272
Dataset/fold9/145683-6-3-0.wav
7273
Dataset/fold9/145683-6-4-0.wav
7274
Dataset/fold9/145683-6-5-0.wav
7275
Dataset/fold9/146249-4-0-0.wav
7276
Dataset/fold9/148166-6-0-0.wav
7295
Dataset/fold9/152588-3-0-0.wav
7297
Dataset/fold9/152588-3-2-0.wav
7298
Dataset/fold9/152588-3-3-0.wav
7305
Dataset/fold9/155130-1-0-0.wav
7306
Dataset/fold9/155130-1-

In [14]:
labels = labels.astype(int)
X_train, Y_train = labels[:-2000], data[:-2000]
X_valid, Y_valid = labels[-2000:], data[-2000:]

C:\Users\gabla\AppData\Local\Temp\ipykernel_18568\679417482.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  X_train, Y_train = labels[:-2000], data[:-2000]
C:\Users\gabla\AppData\Local\Temp\ipykernel_18568\679417482.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  X_valid, Y_valid = labels[-2000:], data[-2000:]


In [15]:
print(Y_train)

[array([1.0000000e+00, 9.2218399e-01, 9.1123486e-01, ..., 8.6038955e-04,
       8.6038955e-04, 8.6038955e-04], dtype=float32), array([1.0000000e+00, 9.4649309e-01, 9.6359754e-01, ..., 9.3066099e-04,
       9.3066099e-04, 9.3066099e-04], dtype=float32), array([0.98027956, 0.90143543, 0.8927545 , ..., 0.00116766, 0.00116766,
       0.00116766], dtype=float32), array([9.964132e-01, 9.198868e-01, 9.289329e-01, ..., 9.916712e-04,
       9.916712e-04, 9.916712e-04], dtype=float32), array([1.        , 0.953722  , 0.9505075 , ..., 0.00139214, 0.00139214,
       0.00139214], dtype=float32), array([9.7467679e-01, 9.1064525e-01, 9.1405910e-01, ..., 5.2952743e-04,
       5.2952743e-04, 5.2952743e-04], dtype=float32), array([9.7006232e-01, 9.2348433e-01, 9.1968095e-01, ..., 1.0690440e-04,
       1.0690440e-04, 1.0690440e-04], dtype=float32), array([1.000000e+00, 9.350806e-01, 9.349226e-01, ..., 9.256595e-04,
       9.256595e-04, 9.256595e-04], dtype=float32), array([1.0000000e+00, 7.1548033e-01, 5.

In [16]:
history = model.fit(X_train, Y_train, epochs=10, validation_data=(X_valid, Y_valid))

ValueError: Data cardinality is ambiguous:
  x sizes: 3930
  y sizes: 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 6747, 